In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import matplotlib.pyplot as plt
import plotly.express as px

/Users/aditkotwal/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
headers = {"User-Agent": "adit29my@gmail.com"} 

## Fetching historical data

In [3]:
def getCIKNumber(ticker, headers=headers):
    ticker = ticker.upper().replace(".", "-")
    ticker_json = requests.get(
        "https://www.sec.gov/files/company_tickers.json", headers=headers
    ).json()

    for company in ticker_json.values():
        if company["ticker"] == ticker:
            cik = str(company["cik_str"]).zfill(10)
            return cik
    raise ValueError(f"Ticker {ticker} not found in SEC database")

In [4]:
def getSubmissionData(ticker, headers=headers):
    cik = getCIKNumber(ticker)
    headers = headers
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    company_json = requests.get(url, headers=headers).json()
    
    return pd.DataFrame(company_json["filings"]["recent"])

In [5]:
def getFilteredFilings(ticker, form, headers=headers):
    company_filings_df = getSubmissionData(
        ticker, headers=headers
    )
    if (form=='ten_k'):
        df = company_filings_df[company_filings_df["form"] == "10-K"]
    elif (form=='ten_q'):
        df = company_filings_df[company_filings_df["form"] == "10-Q"]
    
    df = df.set_index("reportDate")
    accession_df = df["accessionNumber"]
    return accession_df    

In [6]:
def getFacts(ticker, headers=headers):
    cik = getCIKNumber(ticker)
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    company_facts = requests.get(url, headers=headers).json()
    return company_facts

In [7]:
def getFactsDF(ticker, headers=headers):
    facts = getFacts(ticker, headers)
    us_gaap_data = facts["facts"]["us-gaap"]
    df_data = []
    for fact, details in us_gaap_data.items():
        for unit in details["units"]:
            for item in details["units"][unit]:
                row = item.copy()
                row["fact"] = fact
                df_data.append(row)

    df = pd.DataFrame(df_data)
    df["end"] = pd.to_datetime(df["end"])
    df["start"] = pd.to_datetime(df["start"])
    df = df.drop_duplicates(subset=["fact", "end", "val"])
    df.set_index("end", inplace=True)
    labels_dict = {fact: details["label"] for fact, details in us_gaap_data.items()}
    return df, labels_dict

In [8]:
def getAnnualFacts(ticker, headers=headers):
    accession_nums = getFilteredFilings(ticker, form='ten_k')
    df, label_dict = getFactsDF(ticker, headers)
    ten_k = df[df["accn"].isin(accession_nums)]
    ten_k = ten_k[ten_k.index.isin(accession_nums.index)]
    pivot = ten_k.pivot_table(values="val", columns="fact", index="end")
    pivot.rename(columns=label_dict, inplace=True)
    return pivot.T

In [9]:
def getQuarterlyFacts(ticker, headers=headers):
    accession_nums = getFilteredFilings(ticker, form='ten_q')
    df, label_dict = getFactsDF(ticker, headers)
    ten_q = df[df["accn"].isin(accession_nums)]
    ten_q = ten_q[ten_q.index.isin(accession_nums.index)].reset_index(drop=False)
    ten_q = ten_q.drop_duplicates(subset=["fact", "end"], keep="last")
    pivot = ten_q.pivot_table(values="val", columns="fact", index="end")
    pivot.rename(columns=label_dict, inplace=True)
    return pivot.T

In [60]:
def getHistoricalData(ticker):
    quarterlyDF = getQuarterlyFacts(ticker)
    annualDF = getAnnualFacts(ticker)

    # Convert index column to 0th 'fact' column
    quarterlyDF = quarterlyDF.reset_index()
    annualDF = annualDF.reset_index()

    # Merge dataframes based on the new 'fact' column
    combined_df = pd.merge(quarterlyDF, annualDF, on='fact')    

    # Convert the 'fact' column to 'index' again
    combined_df = combined_df.set_index('fact')

    # Sort the columns by date
    combined_df = combined_df.reindex(sorted(combined_df.columns), axis=1)

    return combined_df

## Financial Ratios

In [14]:
def addColumns(df, ratioDictionary):
    for col_name, (required_cols, operation) in ratioDictionary.items():
        if all(col in df.columns for col in required_cols):
            df[col_name] = operation(df)
        else:
            print(f"Skipping '{col_name}': Missing required columns {required_cols}.")
    return df

In [22]:
def calculateRatios(dataDf):
    
    df = pd.DataFrame(dataDf).T
    # df.index = pd.to_datetime(df.index)

    if 'Revenue, Net (Deprecated 2018-01-31)' in df.columns:
        df['Effective Revenue'] = df['Revenue, Net (Deprecated 2018-01-31)'].fillna(df['Revenue from Contract with Customer, Excluding Assessed Tax'])
    else:
        df['Effective Revenue'] = df['Revenue from Contract with Customer, Excluding Assessed Tax']
    
    df['Operating Margin Ratio'] = df['Operating Income (Loss)'] / df['Effective Revenue']
    ratioDictionary = {
        'Gross Margin Ratio': (['Gross Profit', 'Effective Revenue'], lambda df: df['Gross Profit'] / df['Effective Revenue']),
        'Operating Margin Ratio': (['Operating Income (Loss)', 'Effective Revenue'], lambda df: df['Operating Income (Loss)'] / df['Effective Revenue']),
        'Net Profit Margin Ratio': (['Net Income (Loss) Attributable to Parent', 'Effective Revenue'], lambda df: df['Net Income (Loss) Attributable to Parent'] / df['Effective Revenue']),
        'Return on Assets Ratio': (['Net Income (Loss) Attributable to Parent', 'Assets'], lambda df: df['Net Income (Loss) Attributable to Parent'] / df['Assets']),
        'Return on Equity Ratio': (['Net Income (Loss) Attributable to Parent', 'Stockholders\' Equity Attributable to Parent'], lambda df: df['Net Income (Loss) Attributable to Parent'] / df['Stockholders\' Equity Attributable to Parent']),
        'Current Ratio': (['Assets, Current', 'Liabilities, Current'], lambda df: df['Assets, Current'] / df['Liabilities, Current']),
        'Quick Ratio': (['Assets, Current', 'Inventory, Net', 'Liabilities, Current'], lambda df: df['Assets, Current'] - df['Inventory, Net'] / df['Liabilities, Current']),
        'Cash Ratio': (['Cash and Cash Equivalents, at Carrying Value', 'Liabilities, Current'], lambda df: df['Cash and Cash Equivalents, at Carrying Value'] / df['Liabilities']),
        'Debt to Equity (D/E) Ratio': (['Liabilities', 'Stockholders\' Equity Attributable to Parent'], lambda df: df['Liabilities'] / df['Stockholders\' Equity Attributable to Parent']),
        'Debt to Assets Ratio': (['Liabilities', 'Assets'], lambda df: df['Liabilities'] / df['Assets']),
        'Interest Coverage Ratio': (['Operating Income (Loss)', 'Interest Expense'], lambda df: df['Operating Income (Loss)'] / df['Interest Expense']),
        'Equity Ratio': (['Stockholders\' Equity Attributable to Parent', 'Assets'], lambda df: df['Stockholders\' Equity Attributable to Parent'] / df['Assets']),
        'Asset Turnover Ratio': (['Effective Revenue', 'Assets'], lambda df: df['Effective Revenue'] / df['Assets']),
        'Inventory Turnover Ratio': (['Cost of Goods and Services Sold', 'Inventory, Net'], lambda df: df['Cost of Goods and Services Sold'] / df['Inventory, Net']),
        'Receivables Turnover Ratio': (['Effective Revenue', 'Accounts Receivable, after Allowance for Credit Loss, Current'], lambda df: df['Effective Revenue'] / df['Accounts Receivable, after Allowance for Credit Loss, Current']),
        'Days Sales outstanding': (['Receivables Turnover Ratio'], lambda df: 365 / df['Receivables Turnover Ratio']),
        'Days Inventory outstanding': (['Inventory Turnover Ratio'], lambda df: 365 / df['Inventory Turnover Ratio']),
        'Payables Turnover Ratio': (['Cost of Goods and Services Sold', 'Accounts Payable, Current'], lambda df: df['Cost of Goods and Services Sold'] / df['Accounts Payable, Current']),
        'Operating Cash Flow Ratio': (['Net Cash Provided by (Used in) Operating Activities, Continuing Operations', 'Liabilities'], lambda df: df['Net Cash Provided by (Used in) Operating Activities, Continuing Operations'] / df['Liabilities']),
        'Capital Expenditure Coverage Ratio': (['Net Cash Provided by (Used in) Operating Activities, Continuing Operations', 'Payments to Acquire Property, Plant, and Equipment'], lambda df: df['Net Cash Provided by (Used in) Operating Activities, Continuing Operations'] / df['Payments to Acquire Property, Plant, and Equipment']),        
    }

    df_with_ratios = addColumns(df, ratioDictionary)
    df_with_ratios = df_with_ratios.applymap(lambda x: str(x) if isinstance(x, pd.Timestamp) else x)

    return df_with_ratios.T

In [23]:
df = calculateRatios(appleData)
df

/var/folders/sy/sf3lnzz948s6nczk05xz5s640000gn/T/ipykernel_2444/115011178.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_with_ratios = df_with_ratios.applymap(lambda x: str(x) if isinstance(x, pd.Timestamp) else x)


end,2014-03-29,2014-06-28,2014-09-27,2014-12-27,2015-03-28,2015-06-27,2015-09-26,2015-12-26,2016-03-26,2016-06-25,...,2022-06-25,2022-09-24,2022-12-31,2023-04-01,2023-07-01,2023-09-30,2023-12-30,2024-03-30,2024-06-29,2024-09-28
fact,,,,,,,,,,,,,,,,,,,,,
"Accounts Payable, Current",1.891400e+10,2.053500e+10,3.019600e+10,3.800100e+10,2.315900e+10,2.647400e+10,3.549000e+10,3.331200e+10,2.509800e+10,2.631800e+10,...,4.834300e+10,6.411500e+10,5.791800e+10,4.294500e+10,4.669900e+10,6.261100e+10,5.814600e+10,4.575300e+10,4.757400e+10,6.896000e+10
"Accounts Receivable, after Allowance for Credit Loss, Current",9.700000e+09,1.078800e+10,1.746000e+10,1.670900e+10,1.090500e+10,1.037000e+10,1.684900e+10,1.295300e+10,1.222900e+10,1.171400e+10,...,2.180300e+10,2.818400e+10,2.375200e+10,1.793600e+10,1.954900e+10,2.950800e+10,2.319400e+10,2.183700e+10,2.279500e+10,3.341000e+10
"Accrued Income Taxes, Noncurrent",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.069900e+10,1.665700e+10,NaN,NaN,NaN,1.545700e+10,NaN,NaN,NaN,9.254000e+09
"Accrued Liabilities, Current",1.598400e+10,1.526400e+10,1.845300e+10,2.272400e+10,2.282700e+10,2.272400e+10,2.518100e+10,2.403200e+10,2.320800e+10,2.082000e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Accumulated Depreciation, Depletion and Amortization, Property, Plant, and Equipment",1.528600e+10,1.686800e+10,1.839100e+10,2.035500e+10,2.230900e+10,2.439500e+10,2.678600e+10,2.904200e+10,3.084800e+10,3.254300e+10,...,7.151600e+10,7.234000e+10,6.804400e+10,6.966800e+10,7.078700e+10,7.088400e+10,7.251000e+10,7.169700e+10,7.262700e+10,7.344800e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Days Sales outstanding,3.429388e+01,2.799150e+01,5.666865e+01,NaN,3.001550e+01,2.077255e+01,4.312440e+01,6.231344e+01,8.828817e+01,2.533139e+01,...,9.592805e+01,2.608783e+01,7.400072e+01,6.903117e+01,8.723284e+01,2.810029e+01,7.079916e+01,8.782635e+01,9.699774e+01,3.118557e+01
Days Inventory outstanding,2.410141e+01,2.563378e+01,6.863787e+00,1.857629e+01,2.545672e+01,2.490743e+01,6.120288e+00,1.968393e+01,2.717603e+01,2.545768e+01,...,4.212612e+01,8.075698e+00,3.725270e+01,5.166345e+01,5.912028e+01,1.079129e+01,3.671995e+01,4.691803e+01,4.881288e+01,1.264257e+01
Payables Turnover Ratio,1.464471e+00,1.105284e+00,3.717645e+00,1.180443e+00,1.483397e+00,1.130317e+00,3.947281e+00,1.364343e+00,1.220655e+00,9.974922e-01,...,9.737501e-01,3.486641e+00,1.153735e+00,1.230877e+00,9.718409e-01,3.420118e+00,1.113060e+00,1.059646e+00,9.689957e-01,3.050348e+00


## Trends to plot

Labels from dataframe (Ratios):
1. Gross Margin Ratio
2. Operating Margin Ratio
3. Net Profit Margin Ratio
4. Return on Assets Ratio
5. Return on Equity Ratio
6. Earnings Per Share, Basic (already present in the facts, no need to calculate)
7. Earnings Per Share, Diluted (already present in the facts, no need to calculate)
8. Return on Investment (TO-DO)
9. Current Ratio
10. Quick Ratio
11. Cash Ratio
12. Debt to Equity (D/E) Ratio
13. Debt to Assets Ratio
14. Interest Coverage Ratio
15. Equity Ratio
16. Asset Turnover Ratio
17. Inventory Turnover Ratio
18. Receivables Turnover Ratio
19. Days Sales outstanding
20. Days Inventory outstanding
21. Payables Turnover Ratio
22. Cash Conversion Cycle - TODO
23. Price to Earnings (P/E) - TBD
24. Price to Book (P/B) - TBD
25. Price to sales (P/S) - TBD
26. Dividend Yield - TBD
27. Earnings Yield - TBD
28. Operating Cash Flow Ratio
29. Capital Expenditure Coverage Ratio




Miscellaneous trends:
1. Current Assets
2. Total Assets
3. Current Liabilities
4. Total Liabilities